In [8]:
# Import dependencies
import random
import pandas as pd
import matplotlib as plt
import numpy as np
import os
import citipy
import requests
import time
from datetime import datetime


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
coordinates

[(13.666896000406453, 87.1987736872673),
 (72.73390246058545, 131.86753755627944),
 (80.9086185768954, 164.20755884817243),
 (89.14213737792349, -149.25847697372708),
 (78.5649247549267, -72.90621248786071),
 (8.695093733483077, 10.912249245040698),
 (-73.86018227712051, 136.48731029788007),
 (-36.97752429905229, 84.9331809753375),
 (-72.20352642743278, 59.27507721367269),
 (-0.5648654447214909, -74.0378648387775),
 (24.715736707321625, -70.93152326653096),
 (71.34316684858322, -163.78862096585016),
 (1.3167786233102419, -97.39839733865357),
 (24.443579930090024, -172.71687905086134),
 (73.67102321562507, -26.26187751196224),
 (12.815185698567689, 7.415684112641827),
 (-19.93132336901472, 155.72092134248442),
 (41.56572743228324, -155.73680534091264),
 (22.502864042231536, -66.53731910304808),
 (-51.61262239306625, 94.73386687841707),
 (5.504513495563799, -43.55723183727838),
 (62.36708436188732, 4.373418270696476),
 (-56.47340864469083, 58.159668047588156),
 (-9.52143789976094, -101.9

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each lat/long combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

718

In [6]:
# Import the API key.
from config import weather_api_key

# Perform an API call with the OpenWeatherMap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [25]:
# Retrieve the following information from the API call:
# Latitude and longitude
# Maximum temperature
# Percent humidity
# Percent cloudiness
# Wind speed
# Weather description (for example, clouds, fog, light rain, clear sky)

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# # Loop through all the cities in the list.
# for i, city in enumerate(cities):

#     # Group cities in sets of 50 for logging purposes.
#     if (i % 50 == 0 and i >= 50):
#         set_count += 1
#         record_count = 1
     # Create endpoint URL with each city.
#     city_url = url + "&q=" + city.replace(" ","+")

# code from Kourt Bailey on slack

HTTP_STATUS_OK = 200
HTTP_STATUS_NOT_FOUND = 404

for city in cities:
    city_url = url + city.replace(' ', '+')
    response = requests.get(city_url)
    status = response.status_code

    if status == HTTP_STATUS_OK:
        city_data = response.json()
        
        
        
            # continue processing the city...
        print(f"Processing Record {record_count} of Set {set_count} | {city}")
        print(city_url)
        # Add 1 to the record count.
        record_count += 1

        # Run an API request for each of the cities.

        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']

        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })


    elif status == HTTP_STATUS_NOT_FOUND:
        print(f'{city} was not found')
    else:
        raise ValueError(f"Unknown status code {status} for city {city}")


# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------


ValueError: Unknown status code 401 for city yar-sale

In [18]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+") + "&appid=" + weather_api_key
    print(city_url)

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=yarada&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 10 of Set 1 | yarada
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=tiksi&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 11 of Set 1 | tiksi
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=cherskiy&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 12 of Set 1 | cherskiy
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=barrow&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 13 of Set 1 | barrow
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=qaanaaq&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 14 of Set 1 | qaanaaq
http://api.openweathermap.org/data/2.5/weather?units=Imp

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=kimbe&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 52 of Set 1 | kimbe
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=codrington&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 53 of Set 1 | codrington
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=mar+del+plata&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 54 of Set 1 | mar del plata
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=port+elizabeth&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 55 of Set 1 | port elizabeth
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=mataura&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 56 of Set 1 | mataura
http://api.openweather

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=bambous+virieux&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 35 of Set 2 | bambous virieux
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=nobeoka&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 36 of Set 2 | nobeoka
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=te+anau&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 37 of Set 2 | te anau
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=mahebourg&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 38 of Set 2 | mahebourg
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=vorozhba&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 39 of Set 2 | vorozhba
http://api.openweathermap.or

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=galchino&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 28 of Set 3 | galchino
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=henties+bay&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 29 of Set 3 | henties bay
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=provadija&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 30 of Set 3 | provadija
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=araouane&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 31 of Set 3 | araouane
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=sitka&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 32

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=severo-kurilsk&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 21 of Set 4 | severo-kurilsk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=malakal&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 22 of Set 4 | malakal
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=wagga+wagga&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 23 of Set 4 | wagga wagga
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=bitkine&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 24 of Set 4 | bitkine
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=lagoa&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 25 of Set 4 | lagoa
http://api.openweathermap.org/da

...

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=forbes&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 14 of Set 11 | forbes
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=wilmington&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 15 of Set 11 | wilmington
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=pasighat&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 16 of Set 11 | pasighat
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=solovetskiy&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 17 of Set 11 | solovetskiy
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=noumea&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 18 of Set 11 | noumea
http:

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=kirakira&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 7 of Set 12 | kirakira
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=nemuro&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 8 of Set 12 | nemuro
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=nyurba&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 9 of Set 12 | nyurba
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=dosso&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 10 of Set 12 | dosso
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=beyneu&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 11 of Set 12 | beyneu
http://api.openweathermap.org/data/2.5/weather?units=Imp

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=nagua&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 50 of Set 12 | nagua
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=kiunga&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 1 of Set 13 | kiunga
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=maneadero&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 2 of Set 13 | maneadero
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=sechura&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 3 of Set 13 | sechura
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=verdalsora&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 4 of Set 13 | verdalsora
City not found. Skipping...
http://api.openwe

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=bloemfontein&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 42 of Set 13 | bloemfontein
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=bapatla&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 43 of Set 13 | bapatla
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=natitingou&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 44 of Set 13 | natitingou
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=auxerre&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 45 of Set 13 | auxerre
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=alikalia&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 46 of Set 13 | alikalia
http://api.openweathermap.o

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=yumen&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 35 of Set 14 | yumen
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=hue&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 36 of Set 14 | hue
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=vanimo&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 37 of Set 14 | vanimo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=tawau&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 38 of Set 14 | tawau
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=46fb03b52ce7a8ade53dd6b94226cf94&q=urumqi&appid=46fb03b52ce7a8ade53dd6b94226cf94
Processing Record 39 of Set 14 | urumqi
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APP

In [20]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.tail(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
659,Ciudad Bolivar,VE,8.1222,-63.5497,82.67,65,100,1.95,NaN,2022-04-27 22:12:52
660,Mago,RU,53.2514,140.1822,30.78,84,100,14.09,NaN,2022-04-27 22:12:52
661,Ibotirama,BR,-12.1853,-43.2206,82.29,46,76,7.16,NaN,2022-04-27 22:12:52
662,Yamada,JP,39.4667,141.9500,50.95,45,1,4.54,NaN,2022-04-27 22:12:53
663,Mecca,SA,21.4267,39.8261,82.18,68,0,1.88,NaN,2022-04-27 22:12:53
664,Champerico,GT,14.3000,-91.9167,82.94,61,13,11.56,NaN,2022-04-27 22:12:53
665,Shelburne,CA,44.0787,-80.2041,32.13,75,100,16.58,NaN,2022-04-27 22:12:54
666,Dzilam Gonzalez,MX,21.2833,-88.9333,83.10,58,92,11.07,NaN,2022-04-27 22:12:54
667,Sandovo,RU,58.4602,36.4128,26.31,78,2,3.78,NaN,2022-04-27 22:12:54
668,Kerouane,GN,9.2667,-9.0167,79.14,65,95,4.29,NaN,2022-04-27 22:12:55
